In [1]:
!pip install numpy scipy
!pip install ttml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.2 MB/s eta 0:00:00a 0:00:01


In [2]:
import numpy as np

def generate_Fn(n):
    """
    Generates the matrix Fn of size (n+1) x (n+1) according to equation (11).

    Args:
        n (int): Parameter n1, which determines the size of the matrix (n+1) x (n+1).

    Returns:
        numpy.ndarray: The matrix Fn of size (n+1) x (n+1).
    """
    size = n + 1
    F = np.zeros((size, size))

    for i in range(size):
        for j in range(size):
            if i == 0 and j == 0:
                F[i, j] = 1/4  # Top-left corner
            elif i == 0 and j == size - 1:
                F[i, j] = 1/4  # Top-right corner
            elif i == size - 1 and j == 0:
                F[i, j] = 1/4  # Bottom-left corner
            elif i == size - 1 and j == size - 1:
                F[i, j] = 1/4 * np.cos(np.pi * n)  # Bottom-right corner
            elif i == 0:  # First row (except corners)
                F[i, j] = 1/2 * np.cos(np.pi * (j - 1) / n)
            elif j == 0:  # First column (except corners)
                F[i, j] = 1/2
            elif j == size - 1:  # Last column (except corners)
                F[i, j] = 1/2 * np.cos(np.pi * (i - 1))
            elif i == size - 1:  # Last row (except corners)
                F[i, j] = 1/2 * np.cos(np.pi * (j - 1))
            else:  # Inner elements
                F[i, j] = 1/2 * np.cos(np.pi * (i - 1) * (j - 1) / n)

    F = (2 / n) * F
    return F

# Example
if __name__ == "__main__":
    n = 10
    F_matrix = generate_Fn(n)

    print(f"Matrix F_{n} of size {(n+1)}x{(n+1)}:")
    print(F_matrix)

Matrix F_10 of size 11x11:
[[ 5.00000000e-02  1.00000000e-01  9.51056516e-02  8.09016994e-02
   5.87785252e-02  3.09016994e-02  6.12323400e-18 -3.09016994e-02
  -5.87785252e-02 -8.09016994e-02  5.00000000e-02]
 [ 1.00000000e-01  1.00000000e-01  1.00000000e-01  1.00000000e-01
   1.00000000e-01  1.00000000e-01  1.00000000e-01  1.00000000e-01
   1.00000000e-01  1.00000000e-01  1.00000000e-01]
 [ 1.00000000e-01  1.00000000e-01  9.51056516e-02  8.09016994e-02
   5.87785252e-02  3.09016994e-02  6.12323400e-18 -3.09016994e-02
  -5.87785252e-02 -8.09016994e-02 -1.00000000e-01]
 [ 1.00000000e-01  1.00000000e-01  8.09016994e-02  3.09016994e-02
  -3.09016994e-02 -8.09016994e-02 -1.00000000e-01 -8.09016994e-02
  -3.09016994e-02  3.09016994e-02  1.00000000e-01]
 [ 1.00000000e-01  1.00000000e-01  5.87785252e-02 -3.09016994e-02
  -9.51056516e-02 -8.09016994e-02 -1.83697020e-17  8.09016994e-02
   9.51056516e-02  3.09016994e-02 -1.00000000e-01]
 [ 1.00000000e-01  1.00000000e-01  3.09016994e-02 -8.09016



---



# Tensor Train Format (Theory)

We consider a general tensor $\mathcal{X} \in \mathbb{R}^{n_1 \times \dots \times n_d}$ of order $d$.

The TT-ranks of $\mathcal{X}$ form a tuple of integers:

$$\text{rank}_{\text{TT}} = (r_0, \dots, r_d) := (1, \text{rank}(X^{<1>}), \dots, \text{rank}(X^{}), 1)$$

Each entry of the tensor $\mathcal{X}(i_1, \dots, i_d)$ can be expressed as a product of $d$ matrices:

$$\mathcal{X}(i_1,\dots,i_d) = U_1(i_1) U_2(i_2) \dots U_d(i_d)$$

where $U_\mu(i_\mu)$ is a matrix of size $r_{\mu-1} \times r_\mu$. For each $\mu = 1,\dots,d$, the $n_\mu$ matrices $U_\mu(i_\mu)$ for $i_\mu = 1,\dots,n_\mu$ can be grouped into a 3rd-order tensor $\mathbf{U}\mu$ of shape $r{\mu-1} \times n_\mu \times r_\mu$. These are called TT-cores, and by construction we have:

$$\mathcal{X}(i_1,\dots,i_d) = \sum_{k_1=1}^{r_1} \dots \sum_{k_{d-1}=1}^{r_{d-1}} \mathbf{U}_1(1,i_1,k_1) \mathbf{U}_2(k_1,i_2,k_2) \dots \mathbf{U}d(k{d-1},i_d,1)$$

We have a tensor $\mathcal{X} \in \mathbb{R}^{n_1 \times \dots \times n_d}$ in Tensor Train format, which consists of $d$ cores $U_1, \dots, U_d$, where each core $U_k$ corresponds to the $k$-th dimension.

The core $U_k$ for $k = 1,\dots,d$ has shape $(r_{k-1}, n_k, r_k)$, where:
	•	$r_{k-1}$: TT-rank connecting to the previous core (for $k=1$, $r_0=1$),
	•	$n_k$: Size of the $k$-th dimension of the original tensor,
	•	$r_k$: TT-rank connecting to the next core (for $k=d$, $r_d=1$).

Example: If $\mathcal{X}$ has:
	•	Dimensions $n = [n_1, n_2, n_3] = [4, 5, 6]$
	•	TT-ranks $r = [1, 3, 3, 1]$

Then the TT-cores are:
	•	$U_1$: shape $(1, 4, 3)$
	•	$U_2$: shape $(3, 5, 3)$
	•	$U_3$: shape $(3, 6, 1)$



---



# Mode-$\mu$ Matrix Multiplication

The mode-$\mu$ product of a tensor $\mathcal{X} \in \mathbb{R}^{n_1 \times \dots \times n_d}$ with a matrix $M \in \mathbb{R}^{m \times n_\mu}$ is defined as:

$$ \mathcal{Z}(i_1,\dots,i_{\mu-1},j,i_{\mu+1},\dots,i_d) = \sum_{i_\mu=1}^{n_\mu} \mathcal{X}(i_1,\dots,i_d) M(j,i_\mu), \quad j = 1, \dots, m $$

This operation is denoted by $\mathcal{Z} = \mathcal{X} \times_\mu M$. The resulting tensor $\mathcal{Z} \in \mathbb{R}^{n_1 \times \dots \times n_{\mu-1} \times m \times n_{\mu+1} \times \dots \times n_d}$.

This means that the matrix $M$ “acts” on the $\mu$-th mode of the tensor, replacing its dimension $n_\mu$ with $m$ (the number of rows in $M$), while leaving the other dimensions unchanged.

In our case, we will use mode-$\mu$ matrix multiplication to compute the product $\mathcal{P} \times_1 F_n$, where $F_n \in \mathbb{R}^{(n+1) \times (n+1)}$ and $\mathcal{P}$ is the tensor resulting from tensor completion, represented in TT format.

This operation is performed with the goal of obtaining the tensor $\mathcal{C}$ of Chebyshev interpolation coefficients. In our specific case ($\mu = 1$):
	•	$\mathcal{P} \in \mathbb{R}^{n_1 \times \dots \times n_d}$ is the tensor in TT format.
	•	$F_{n_1} \in \mathbb{R}^{m \times n_1}$ is the matrix generated by the function above.
	•	The result $\mathcal{Z} = \mathcal{P} \times_1 F_{n_1}$ is a tensor
$\mathcal{Z} \in \mathbb{R}^{m \times n_2 \times \dots \times n_d}$.
	•	The mode-1 multiplication is defined as:

$$
\mathcal{Z}(j,i_2,\dots,i_d) = \sum_{i_1=1}^{n_1} \mathcal{P}(i_1,i_2,\dots,i_d) \cdot F_n(j,i_1), \quad j=1,\dots,m
$$

Note: This is the definition in the dense case. What it does is transform the first dimension of $\mathcal{P}$ from size $n_1$ to $m$, as if $F_{n_1}$ were multiplying each mode-1 “fiber” of $\mathcal{P}$.

**Application: Mode-1 Matrix Multiplication in TT Format**

When multiplying $\mathcal{P}$ by $F_{n_1}$ along mode-1, we aim to obtain $\mathcal{Z}$ in TT format. This operation affects only the first dimension, represented by the first core $\mathbf{U}_1$ (a 3rd-order tensor).

So we modify only $\mathbf{U}_1$ while keeping the other TT-cores unchanged.
	•	$\mathbf{U}_1$ has shape $(1, n_1, r_1)$
	•	$F_{n_1}$ has shape $(m, n_1)$
	•	The new TT-core will have shape $(1, m, r_1)$



---



In order for the algorithm to work properly, we must assume that
$\mathcal{P} \in \mathbb{R}^{(n_1+1) \times (n_2+1) \times \dots \times (n_d+1)}$.
This ensures that we can apply mode-2 matrix multiplication of each TT-core $\mathbf{U}\mu$ of $\mathcal{P}$ by the matrix $F{n_\mu} \in \mathbb{R}^{(n_\mu+1) \times (n_\mu+1)}$ without issues.

In [4]:
import numpy as np

def mode_mu_multiply(X, M, mu):
    """
    Performs mode-μ multiplication between a tensor X and a matrix M.

    Args:
        X (numpy.ndarray): Input tensor of shape (n_1, ..., n_d).
        M (numpy.ndarray): Matrix of shape (m, n_mu), where n_mu is the size of the μ-th dimension.
        mu (int): Index of the dimension for multiplication (1-based, internally adjusted to 0-based).

    Returns:
        numpy.ndarray: Resulting tensor Z of shape (n_1, ..., n_{mu-1}, m, n_{mu+1}, ..., n_d).

    Raises:
        ValueError: If mu is out of bounds or M's dimensions do not match n_mu.
    """
    dims = X.shape
    d = len(dims)

    if mu < 1 or mu > d:
        raise ValueError(f"mu must be between 1 and {d}")
    mu -= 1

    n_mu = dims[mu]
    m, n_mu_M = M.shape
    if n_mu_M != n_mu:
        raise ValueError(f"Second dimension of M ({n_mu_M}) must match n_{mu+1} ({n_mu})")

    new_dims = list(dims)
    new_dims[mu] = m

    # Permute to move mu-th axis to the end
    axes_order = list(range(d))
    axes_order.pop(mu)
    axes_order.append(mu)
    X_permuted = np.transpose(X, axes_order)

    # Reshape to matrix
    X_mat = X_permuted.reshape(-1, n_mu)

    # Matrix multiplication
    Z_mat = np.dot(X_mat, M.T)

    # Reshape back to tensor
    Z_permuted = Z_mat.reshape(*new_dims[0:mu], m, *new_dims[mu+1:])
    inverse_axes = list(range(mu)) + [d-1] + list(range(mu, d-1))
    Z = np.transpose(Z_permuted, axes=inverse_axes)

    return Z

In [5]:
# Example of usage
if __name__ == "__main__":
    # Create a sample 3D tensor X
    n1, n2, n3 = 4, 5, 3  # Dimensions: (n_1, n_2, n_3)
    X = np.random.rand(n1, n2, n3)  # Random tensor of shape (4, 5, 3)

    # Create a matrix M to multiply along mode mu=2
    m = 3  # New dimension replacing n_2
    M = np.random.rand(m, n2)  # Matrix of shape (m, n_2) = (3, 5)
    mu = 2  # Mode-2 multiplication (1-based index)

    # Perform the mode-μ multiplication
    Z = mode_mu_multiply(X, M, mu)

    # Display results
    print("Shape of X:", X.shape)  # Should be (4, 5, 3)
    print("Shape of M:", M.shape)  # Should be (3, 5)
    print("Shape of Z:", Z.shape)  # Should be (4, 3, 3)
    print("First elements of Z:\n", Z[:2, :2, :2])  # Show a small slice of Z

Shape of X: (4, 5, 3)
Shape of M: (3, 5)
Shape of Z: (4, 3, 3)
First elements of Z:
 [[[0.19220043 0.72448725]
  [0.59425929 1.16160491]]

 [[1.02139197 1.38547352]
  [1.71538183 1.35754243]]]




---



If the tensor $\mathcal{X}$ is in Tensor Train (TT) decomposition, then it is straightforward to obtain a TT decomposition of $\mathcal{Z}$ by performing a mode-2 matrix multiplication of the TT-core $\mathbf{U}_\mu$ with $M$.

In [6]:
import numpy as np
from ttml.tensor_train import TensorTrain

# Assuming the function mode_mu_multiply is already defined
def mode_mu_multiply(X, M, mu):
    """
    Performs mode-μ matrix multiplication between a dense tensor X and a matrix M.

    Args:
        X (numpy.ndarray): Input tensor of shape (n_1, ..., n_d).
        M (numpy.ndarray): Matrix of shape (m, n_mu), where n_mu is the size of the mu-th dimension.
        mu (int): Index of the dimension for multiplication (1-based, internally adjusted to 0-based).

    Returns:
        numpy.ndarray: Resulting tensor Z of shape (n_1, ..., n_{mu-1}, m, n_{mu+1}, ..., n_d).
    """
    dims = X.shape  # (n_1, n_2, ..., n_d)
    d = len(dims)   # Number of dimensions

    # Adjust mu to 0-based index
    if mu < 1 or mu > d:
        raise ValueError(f"mu must be between 1 and {d}")
    mu -= 1

    # Check that second dimension of M matches n_mu
    n_mu = dims[mu]
    m, n_mu_M = M.shape
    if n_mu_M != n_mu:
        raise ValueError(f"The second dimension of M ({n_mu_M}) must match n_{mu+1} ({n_mu})")

    # Create the new shape for Z
    new_dims = list(dims)
    new_dims[mu] = m  # Replace n_mu with m in dimension mu

    # Permute X to move the mu-th dimension to the last position
    axes_order = list(range(d))
    axes_order.pop(mu)
    axes_order.append(mu)
    X_permuted = np.transpose(X, axes_order)

    # Reshape X for matrix multiplication
    X_mat = X_permuted.reshape(-1, n_mu)

    # Perform matrix multiplication: Z_mat shape will be (product of other dims × m)
    Z_mat = np.dot(X_mat, M.T)  # M.T because we want M(j, i_mu), not M(i_mu, j)

    # Reshape Z_mat back to tensor shape
    Z_permuted = Z_mat.reshape(*new_dims[0:mu], m, *new_dims[mu+1:])

    # Restore original axis order
    inverse_axes = list(range(mu)) + [d - 1] + list(range(mu, d - 1))
    Z = np.transpose(Z_permuted, axes=inverse_axes)

    return Z

def mode_mu_tt_multiply(P, F_n_mu, mu):
    """
    Performs mode-μ multiplication between a TT tensor P and a matrix F_n_mu,
    using mode_mu_multiply on the TT cores and returning Z in TT format.

    Args:
        P (TensorTrain): TT-format tensor with dimensions [(n_1+1), ..., (n_d+1)].
        F_n_mu (numpy.ndarray): Matrix of shape (n_mu + 1, n_mu + 1).
        mu (int): Index of the dimension for multiplication (1-based).

    Returns:
        TensorTrain: Resulting TT tensor Z = P ×_mu F_n_mu.
    """
    if mu < 1 or mu > len(P.dims):
        raise ValueError(f"mu must be between 1 and {len(P.dims)}")
    mu -= 1

    # Check compatibility of matrix dimensions
    n_mu_plus_1 = P.dims[mu]
    m, n_mu_F = F_n_mu.shape
    if n_mu_F != n_mu_plus_1 or m != n_mu_plus_1:
        raise ValueError(f"Dimensions of F_n_mu ({m}x{n_mu_F}) do not match n_mu+1 ({n_mu_plus_1})")

    # Extract TT-core U_mu of shape (r_{mu-1}, n_mu + 1, r_mu)
    core_mu = P.cores[mu]
    r_prev, n, r_next = core_mu.shape

    # Reshape for mode-1 multiplication: (r_prev * n, r_next)
    reshaped_core = core_mu.reshape(r_prev * n, r_next)

    # Perform dense mode-1 multiplication
    temp_tensor_2d = reshaped_core
    Z_temp = mode_mu_multiply(temp_tensor_2d, F_n_mu, 1)

    # Reshape result to new TT-core: (r_prev, m, r_next)
    new_core_mu = Z_temp.reshape(r_prev, m, r_next)

    # Replace the mu-th core in the TT
    new_cores = [core.copy() for core in P.cores]
    new_cores[mu] = new_core_mu

    return TensorTrain(new_cores)

In [8]:
# Example of usage
if __name__ == "__main__":
    # Set up an example TT tensor
    n = 4  # Base size for each dimension
    d = 3  # 3 dimensions
    dims = [n + 1] * d  # Dimensions: [5, 5, 5]
    initial_rank = 3  # Consistent initial rank

    # Create initial TT cores for P
    cores = []
    for k in range(d):
        if k == 0:
            cores.append(np.random.rand(1, dims[k], initial_rank))  # (1, 5, 3)
        elif k == d - 1:
            cores.append(np.random.rand(initial_rank, dims[k], 1))  # (3, 5, 1)
        else:
            cores.append(np.random.rand(initial_rank, dims[k], initial_rank))  # (3, 5, 3)

    P = TensorTrain(cores)

    # Generate matrix F_n_mu for mu = 1 (n_mu = n = 4)
    mu = 1  # Mode-1
    F_n_mu = generate_Fn(n)  # Matrix (n + 1) x (n + 1) = (5, 5)

    # Perform mode-1 multiplication
    Z = mode_mu_tt_multiply(P, F_n_mu, mu)

    # Display results
    print("Original dimensions of P:", P.dims)  # [5, 5, 5]
    print("Dimensions of Z:", Z.dims)  # [5, 5, 5] (same shape, since F_n_mu is square)
    print("Original TT ranks of P:", P.tt_rank)  # [1, 3, 3, 1]
    print("TT ranks of Z:", Z.tt_rank)  # Should be [1, 3, 3, 1] or adjusted

Original dimensions of P: (5, 5, 5)
Dimensions of Z: (5, 5, 5)
Original TT ranks of P: (3, 3)
TT ranks of Z: (3, 3)




---



# Algorithm 3. Efficient computation of $\mathcal{C}$ (TT tensor of Chebyshev interpolation coefficients)

Input: Tensor $\mathcal{P}$ in TT format, containing the function values on the Chebyshev grid.

Output: Tensor (in TT format) $\mathcal{C} \in \mathbb{R}^{(n_1+1)\times(n_2+1)\times\dots\times(n_d+1)}$, defined by

$$\mathcal{C}(i_1,i_2,\dots,i_d) = c_{i_1-1,i_2-1,\dots,i_d-1}$$

for $i_j = 1,\dots,n_j+1$ and $j = 1,\dots,d$.

	1.	Compute $F_{n_1}$ using the function generate_Fn.
	2.	$\mathcal{C} \leftarrow \mathcal{P} \times_1 F_{n_1}$
	3.	For $m = 2,\dots,d$:
  3.1 Compute $F_{n_m}$
  3.2 $\mathcal{C} \leftarrow \mathcal{C} \times_m F_{n_m}$
	4.	End for

Important: If $n_1 = \dots = n_d$ (the usual case), the algorithm simplifies because $F_n$ only needs to be computed once. The specific structure of the matrices $F_{n_i}$ allows for the use of a Fast-Fourier-Transform-based algorithm that computes each mode-$\mu$ multiplication more efficiently.

In [7]:
import numpy as np
from ttml.tensor_train import TensorTrain

def mode_mu_multiply(X, M, mu):
    """
    Performs mode-μ multiplication between a dense tensor X and a matrix M.

    Args:
        X (numpy.ndarray): Input tensor of shape (n_1, ..., n_d).
        M (numpy.ndarray): Matrix of shape (m, n_mu).
        mu (int): Dimension index (1-based, internally adjusted to 0-based).

    Returns:
        numpy.ndarray: Resulting tensor Z.
    """
    dims = X.shape
    d = len(dims)
    if mu < 1 or mu > d:
        raise ValueError(f"mu must be between 1 and {d}")
    mu -= 1

    n_mu = dims[mu]
    m, n_mu_M = M.shape
    if n_mu_M != n_mu:
        raise ValueError(f"Second dimension of M ({n_mu_M}) must match n_{mu+1} ({n_mu})")

    new_dims = list(dims)
    new_dims[mu] = m

    axes_order = list(range(d))
    axes_order.pop(mu)
    axes_order.append(mu)
    X_permuted = np.transpose(X, axes_order)

    X_mat = X_permuted.reshape(-1, n_mu)
    Z_mat = np.dot(X_mat, M.T)

    Z_permuted = Z_mat.reshape(*new_dims[0:mu], m, *new_dims[mu+1:])
    inverse_axes = list(range(mu)) + [d - 1] + list(range(mu, d - 1))
    Z = np.transpose(Z_permuted, axes=inverse_axes)

    return Z

def mode_mu_tt_multiply(P, F_n_mu, mu):
    """
    Performs mode-μ multiplication between a TT tensor P and a matrix F_n_mu,
    using mode_mu_multiply on the cores and returning Z in TT format.

    Args:
        P (TensorTrain): TT-format tensor with shape [(n_1+1), ..., (n_d+1)].
        F_n_mu (numpy.ndarray): Matrix of shape (n_mu + 1, n_mu + 1).
        mu (int): Dimension index for multiplication (1-based).

    Returns:
        TensorTrain: Resulting TT tensor Z = P ×_mu F_n_mu.
    """
    if mu < 1 or mu > len(P.dims):
        raise ValueError(f"mu must be between 1 and {len(P.dims)}")
    mu -= 1

    n_mu_plus_1 = P.dims[mu]
    m, n_mu_F = F_n_mu.shape
    if n_mu_F != n_mu_plus_1 or m != n_mu_plus_1:
        raise ValueError(f"Dimensions of F_n_mu ({m}x{n_mu_F}) do not match n_mu+1 ({n_mu_plus_1})")

    core_mu = P.cores[mu]
    r_prev, n, r_next = core_mu.shape

    reshaped_core = core_mu.reshape(n, r_prev * r_next)

    temp_tensor_2d = reshaped_core
    Z_temp = mode_mu_multiply(temp_tensor_2d, F_n_mu, 1)

    new_core_mu = Z_temp.reshape(r_prev, m, r_next)

    new_cores = [core.copy() for core in P.cores]
    new_cores[mu] = new_core_mu

    return TensorTrain(new_cores)

def generate_Fn(n):
    """
    Generates the matrix Fn of shape (n+1) x (n+1) as defined in equation (11).

    Args:
        n (int): Parameter ni determining the size of the matrix.

    Returns:
        numpy.ndarray: Matrix Fn of shape (n+1) x (n+1).
    """
    size = n + 1
    F = np.zeros((size, size))
    for i in range(size):
        for j in range(size):
            if (i == 0 and j == 0) or (i == n and j == n):
                F[i, j] = 1/4
            elif i == 0 or i == n or j == 0 or j == n:
                F[i, j] = 1/2 * np.cos(np.pi * i * j / n)
            else:
                F[i, j] = 1/2 * np.cos(np.pi * (i - 1) * (j - 1) / n)
    F = (2 / n) * F
    return F

def compute_C_efficient(P, n, d):
    """
    Implements Algorithm 3 to efficiently compute the Chebyshev coefficient tensor C in TT format,
    applying mode-μ multiplications with matrices F_n.

    Args:
        P (TensorTrain): Initial TT tensor with dimensions [(n+1), ..., (n+1)].
        n (int): Base size for each dimension (n_i = n).
        d (int): Number of dimensions.

    Returns:
        TensorTrain: TT tensor C with Chebyshev coefficients.
    """
    F_n1 = generate_Fn(n)
    C = mode_mu_tt_multiply(P, F_n1, 1)

    for m in range(2, d + 1):
        F_nm = generate_Fn(n)
        C = mode_mu_tt_multiply(C, F_nm, m)

    return C

# Example of usage
if __name__ == "__main__":
    n = 4  # Base size for each dimension
    d = 3  # 3 dimensions
    dims = [n + 1] * d  # [5, 5, 5]
    initial_rank = 3

    cores = []
    for k in range(d):
        if k == 0:
            cores.append(np.random.rand(1, dims[k], initial_rank))  # (1, 5, 3)
        elif k == d - 1:
            cores.append(np.random.rand(initial_rank, dims[k], 1))  # (3, 5, 1)
        else:
            cores.append(np.random.rand(initial_rank, dims[k], initial_rank))  # (3, 5, 3)

    P = TensorTrain(cores)
    C = compute_C_efficient(P, n, d)

    print("Dimensions of P:", P.dims)
    print("Dimensions of C:", C.dims)
    print("TT ranks of P:", P.tt_rank)
    print("TT ranks of C:", C.tt_rank)

Dimensions of P: (5, 5, 5)
Dimensions of C: (5, 5, 5)
TT ranks of P: (3, 3)
TT ranks of C: (3, 3)
